<a href="https://colab.research.google.com/github/DarkDomian/TechTranslate/blob/main/translate_tech_documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Required Libraries
!pip install transformers
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.8 MB/s eta 0:00:00


In [ ]:
# Install tokenizer and model itself
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-en-ru"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Function to translate text
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    translated_tokens = model.generate(**inputs)
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [ ]:
# Import the file for translating
from google.colab import files

# Create a file upload form
uploaded = files.upload()

for filename in uploaded.keys():
    with open(filename, "r", encoding="utf-8") as f:
        md_content = f.read()

    output_file = open(f"NLP_{filename}", "w", encoding="utf-8")

Saving lifecycle-of-reactive-effects.md to lifecycle-of-reactive-effects.md


In [ ]:
# NEW hadle for translation
import re

sections = [part for part in re.split(r'(```.*?```|^(#{1,6}\s*)(.*?)(\s*\{.*?\})$|\n{1,2}|<.*?>|`.*?`)', md_content, flags=re.DOTALL) if part]

for section in sections:
    if re.match(r'(```.*?```|\n{1,2}|<.*?>|`.*?`)', section, flags=re.DOTALL):
        output_file.write(section)
    elif re.match(r'^(#{1,6}\s*)(.*?)(\s*\{.*?\})$', section):
        updated_section = re.sub(
            r'^(#{1,6}\s*)(.*?)(\s*\{.*?\})$',
            lambda match: f"{match.group(1)}{translate_text(match.group(2))}{match.group(3)}",
            section
        )
        output_file.write(updated_section)
    else:
        output_file.write(translate_text(section))

output_file.close()

In [ ]:
# OLD version of hadle the translation
inside_code_block = False

for line in md_content:
    if line.startswith("\n"):
        output_file.write("\n")
        continue
    if line.startswith("```"):
        inside_code_block = not inside_code_block
        output_file.write(translate_text(line.strip()) + "\n")
        continue
    if inside_code_block:
        if bool(re.search(r"//", line)):
            line_parts = line.split("//")
            output_file.write(line_parts[0] + ' // ' + translate_text(line_parts[1]))
        else:
            output_file.write(line)
        continue
    if line.startswith("<") and line.endswith(">\n"):
        output_file.write(line)

    output_file.write(translate_text(line.strip()) + "\n")

output_file.close()